# Solving Large-Scale Linear Programming Models
## Mehmet Gönen
## November 9, 2022

In [1]:
# load libraries
import numpy as np
import scipy.sparse as sp
import cplex as cp

In [2]:
def linear_programming(direction, A, senses, b, c, l, u):
    # create an empty optimization problem
    prob = cp.Cplex()

    # add decision variables to the problem including their coefficients in objective and ranges
    prob.variables.add(obj = c.tolist(), lb = l.tolist(), ub = u.tolist())

    # define problem type
    if direction == "maximize":
        prob.objective.set_sense(prob.objective.sense.maximize)
    else:
        prob.objective.set_sense(prob.objective.sense.minimize)

    # add constraints to the problem including their directions and right-hand side values
    prob.linear_constraints.add(senses = senses.tolist(), rhs = b.tolist())

    # add coefficients for each constraint
    row_indices, col_indices = A.nonzero()
    prob.linear_constraints.set_coefficients(zip(row_indices.tolist(),
                                                 col_indices.tolist(),
                                                 A.data.tolist()))

    # solve the problem
    prob.solve()

    # check the solution status
    print(prob.solution.get_status())
    print(prob.solution.status[prob.solution.get_status()])

    # get the solution
    x_star = prob.solution.get_values()
    obj_star = prob.solution.get_objective_value()

    return(x_star, obj_star)

## The multicommodity flow problem

Let $\mathcal{G} = (\mathcal{V}, \mathcal{A})$ be a directed graph, and let $\mathcal{K}$ be a set of commodities. For each link $(i, j) \in \mathcal{A}$ and each commodity $k$, associate a cost per unit of flow, designated by $c_{ij}^{k}$. The demand (or supply) at each node $i \in \mathcal{V}$ for commodity $k$ is designated as $b_{i}^{k}$, where $b_{i}^{k} > 0$ denotes a supply node and $b_{i}^{k} < 0$ denotes a demand node. Define decision variables $x_{ij}^{k}$ that denote the amount of commodity $k$ sent from node $i$ and node $j$. The amount of total flow, across all commodities, that can be sent across each link is bounded above by $u_{ij}$.

The problem can be modeled as a linear programming problem as follows:
\begin{align*}
\mbox{minimize} \;\;& \sum\limits_{(i, j) \in \mathcal{A}} \sum\limits_{k \in \mathcal{K}} c_{ij}^{k} x_{ij}^{k} \\
\mbox{subject to:} \;\;& \sum\limits_{(i, j) \in \mathcal{A}} x_{ij}^{k} - \sum\limits_{(j, i) \in \mathcal{A}} x_{ji}^{k} = b_{i}^{k} \;\;\;\; i \in \mathcal{V} \;\;\;\; k \in \mathcal{K} \\
\;\;& \sum\limits_{k \in \mathcal{K}} x_{ij}^{k} \leq u_{ij} \;\;\;\; (i, j) \in \mathcal{A} \\
\;\;& x_{ij}^{k} \geq 0 \;\;\;\; (i, j) \in \mathcal{A} \;\;\;\; k \in \mathcal{K}.
\end{align*}
In this formulation, the balance constraints ensure that the flow of commodities leaving each supply node and entering each demand node are balanced. The capacity constraints limit the total flow across all commodities on each arc.

In this formulation, the balance constraints ensure that the flow of commodities leaving each supply node and entering each demand node are balanced. The capacity constraints limit the total flow across all commodities on each arc.

In [3]:
def multicommodity_flow_problem(costs_file, capacities_file, flows_file):
    costs = np.loadtxt(costs_file)
    capacities = np.loadtxt(capacities_file)
    flows = np.loadtxt(flows_file)

    E = capacities.shape[0]
    K = costs.shape[0] // capacities.shape[0]
    V = flows.shape[0] // K

    c = costs[:, 3]
    senses = np.concatenate((np.repeat("E", V * K), np.repeat("L", E)))
    b = np.concatenate((flows[:, 2], capacities[:, 2]))
    l = np.repeat(0, E * K)
    u = np.repeat(cp.infinity, E * K)

    aij = np.concatenate((np.repeat([+1.0, -1.0], E * K), np.repeat(+1.0, E * K)))
    row = np.concatenate((costs[:, 0].astype(int) - 1 + (costs[:, 2].astype(int) - 1) * V, 
                          costs[:, 1].astype(int) - 1 + (costs[:, 2].astype(int) - 1) * V,
                          V * K + np.repeat(range(E), K)))
    col = np.concatenate((range(E * K), range(E * K), np.array(range(E * K)).reshape(K, E).T.flatten()))
    A = sp.csr_matrix((aij, (row, col)), shape = (V * K + E, E * K))

    #import matplotlib.pyplot as plt
    #plt.figure(figsize = (6.4, 7.0))
    #plt.spy(A, marker = "o", markersize = 6)
    #plt.show()

    x_star, obj_star = linear_programming("minimize", A, senses, b, c, l, u)
    return(x_star, obj_star)

In [4]:
x_star, obj_star = multicommodity_flow_problem("costs2.txt", "capacities2.txt", "flows2.txt")
print(x_star)
print(obj_star)

Version identifier: 22.1.0.0 | 2022-03-27 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 3 rows and 0 columns.
Aggregator did 4 substitutions.
Reduced LP has 10 rows, 10 columns, and 30 nonzeros.
Presolve time = 0.00 sec. (0.02 ticks)
Symmetry aggregator did 8 additional substitutions.

Iteration log . . .
Iteration:     1   Dual objective     =           820.000000

Dual crossover.
  Dual:  Fixed no variables.
  Primal:  Fixing 1 variable.
        0 PMoves:  Infeasibility  0.00000000e+00  Objective  9.80000000e+02
  Primal:  Pushed 0, exchanged 1.
1
optimal
[0.0, 30.0, 20.0, 40.0, 70.0, 0.0, 10.0, 0.0, 50.0, 0.0, 40.0, 90.0, 0.0, 30.0]
980.0


In [5]:
x_star, obj_star = multicommodity_flow_problem("costs5.txt", "capacities5.txt", "flows5.txt")
print(x_star)
print(obj_star)

Version identifier: 22.1.0.0 | 2022-03-27 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 3 rows and 0 columns.
Aggregator did 10 substitutions.
Reduced LP has 19 rows, 25 columns, and 75 nonzeros.
Presolve time = 0.00 sec. (0.04 ticks)
Symmetry aggregator did 32 additional substitutions.

Iteration log . . .
Iteration:     1   Dual objective     =          2050.000000

Dual crossover.
  Dual:  Fixed no variables.
  Primal:  Fixing 4 variables.
        3 PMoves:  Infeasibility  0.00000000e+00  Objective  2.45000000e+03
        0 PMoves:  Infeasibility  0.00000000e+00  Objective  2.45000000e+03
  Primal:  Pushed 1, exchanged 3.
1
optimal
[0.0, 30.0, 20.0, 40.0, 70.0, 0.0, 10.0, 0.0, 50.0, 0.0, 40.0, 90.0, 0.0, 30.0, 0.0, 20.0, 30.0, 40.0, 60.0, 0.0, 0.0, 0.0, 50.0, 0.0, 40.0, 90.0, 0.0, 30.0, 0.0, 50.0, 0.0, 40.0, 90.0, 0.0, 30.0]
2450.0


In [6]:
x_star, obj_star = multicommodity_flow_problem("costs25.txt", "capacities25.txt", "flows25.txt")
print(x_star)
print(obj_star)

Version identifier: 22.1.0.0 | 2022-03-27 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 3 rows and 0 columns.
Aggregator did 50 substitutions.
Reduced LP has 79 rows, 125 columns, and 375 nonzeros.
Presolve time = 0.00 sec. (0.19 ticks)
Symmetry aggregator did 192 additional substitutions.

Iteration log . . .
Iteration:     1   Dual objective     =         10250.000000

Dual crossover.
  Dual:  Fixed no variables.
  Primal:  Fixing 24 variables.
       23 PMoves:  Infeasibility  0.00000000e+00  Objective  1.22500000e+04
        0 PMoves:  Infeasibility  0.00000000e+00  Objective  1.22500000e+04
  Primal:  Pushed 8, exchanged 16.
1
optimal
[0.0, 40.0, 10.0, 40.0, 80.0, 0.0, 20.0, 0.0, 20.0, 30.0, 40.0, 60.0, 0.0, 0.0, 0.0, 50.0, 0.0, 40.0, 90.0, 0.0, 30.0, 0.0, 50.0, 0.0, 40.0, 90.0, 0.0, 30.0, 0.0, 20.0, 30.0, 40.0, 60.0, 0.0, 0.0, 0.0, 50.0, 0.0, 40.0, 90.0, 0.0, 30.0, 0.0, 50.0, 0.0, 40.0, 90.0, 0.0, 30.0, 0.0, 20.0, 3